In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from translate import Translator

In [3]:
url = 'https://pl.wikipedia.org/wiki/Ptaki_Polski'

response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

tables = soup.find_all('table', class_='wikitable')

rows = []

for table in tables:
    for row in table.find_all('tr')[1:]:
        cells = row.find_all(['th', 'td'])
        if cells:
            if not any("Rodzina" in cell.text for cell in cells):
                row_data = [cell.text.strip() for cell in cells[:-1]]
                rows.append(row_data)

df = pd.DataFrame(rows, columns=["Nazwa polska", "Nazwa łacińska", "Kat. AERC", "Status gatunku w Polsce", "Ochrona w Polsce"])


In [4]:
translation_dict = {
    "Nazwa polska": "Polish name",
    "Nazwa łacińska": "Latin name",
    "Kat. AERC": "AERC category",
    "Status gatunku w Polsce": "Status in Poland",
    "Ochrona w Polsce": "Protection in Poland",
    "Grafika": "Image"
}

df.rename(columns=translation_dict, inplace=True)

In [5]:
df

,Polish name,Latin name,AERC category,Status in Poland,Protection in Poland
0,sterniczka,Oxyura leucocephala,A,zalatuje sporadycznie,pod ścisłą ochroną
1,łabędź niemy,Cygnus olor,A,nielicznie lęgowy,pod ścisłą ochroną
2,łabędź czarnodzioby,Cygnus columbianus,A,bardzo nielicznie przelotny,pod ścisłą ochroną
3,łabędź krzykliwy,Cygnus cygnus,A,"skrajnie nieliczny lęgowy, nielicznie zimujący...",pod ścisłą ochroną
4,bernikla obrożna,Branta bernicla,A,zalatuje rzadko,pod ścisłą ochroną
...,...,...,...,...,...
472,trznadel rudogłowy,Granativora bruniceps,A,zalatuje wyjątkowo,
473,trznadel szarogłowy,Emberiza spodocephala,A,zalatuje wyjątkowo,
474,potrzos,Schoeniclus schoeniclus,A,bardzo licznie lęgowy,pod ścisłą ochroną
475,junko,Junco hyemalis,A,zalatuje wyjątkowo,pod ścisłą ochroną


In [6]:
def translate_polish_to_english(text):
    translator = Translator(from_lang='pl',to_lang="en")
    translated_text = translator.translate(text)
    return translated_text

In [ ]:
# Change latin name to be compatible with xeno-canto API
df.loc[df['Polish name'] == 'gołąb miejski', 'Latin name'] = 'Columba livia domestica'

In [17]:
df.to_csv('birds.csv', index=False)

In [8]:
df["English name"] = df["Polish name"].apply(translate_polish_to_english)